# Convert to h3

Now that it's preprocessed, I want to hex the parquet files 

In [ ]:
import ibis
from ibis import _
from h3_utils import *
from cng.utils import *
from cng.h3 import *

duckdb_install_h3()
con = ibis.duckdb.connect(extensions = ["spatial", "h3"])
con.raw_sql("SET THREADS=100;")
set_secrets(con)
    
#get fips code for each state
fips_url = 'https://www2.census.gov/geo/docs/reference/codes2020/national_state2020.txt'
fips_codes = con.read_csv(fips_url).filter(_.STATE.notin(['AS','GU','MP','PR','UM','VI'])).select("STATEFP").execute().values.flatten().tolist()
z = 8 
chamber = 'upper' # upper or lower 

for state in fips_codes:
        chamber_acronym = 'sldu' if chamber == 'upper' else 'sldl' #sldu for upper, sldl for lower 
        save_url = f's3://public-census/2024/sld/{chamber}/z{z}/tl_2024_{state}_{chamber_acronym}_z{z}.parquet'
        if not minio_file_exists(save_url):
            print(f'Processing for {state}')
            url = f's3://public-census/2024/sld/{chamber}/tl_2024_{state}_{chamber_acronym}.parquet'
            df = con.read_parquet(url).mutate(geom = _.geom.convert('EPSG:4269','EPSG:4326'))
            convert_to_h3(z, save_url, df)
        else: 
            print(f'File already exists for {state}')
    
alert() #makes noise to notify me that it's done processing 